In [1]:
import numpy as np
import pandas as pd
from learning import train_with_early_stopping, train_fn, eval_fn
from learning import CustomDataset, dataset_split, load_model
from learning.visualization import sent_length_vis, plot_training_progress
from transformers import ElectraTokenizer, ElectraForSequenceClassification

# KOELECTRA 모델 경로
modelpath = "./koelectra-base-v3-discriminator"

In [ ]:
# 데이터 로드
dataset = pd.read_csv("./a.csv", encoding='UTF-8')
dataset.dropna(axis=0, inplace=True) # 결측치 제거
dataset.drop_duplicates(inplace=True) # 중복 제거
dataset.reset_index(drop=True, inplace=True) # 인덱스 리셋

print(dataset.isnull().sum()) # 결측치 확인
print(dataset['sentiment'].value_counts()) # 라벨 갯수분포 확인
dataset

In [3]:
# 문장과 라벨 리스트화
sentences = dataset['sentence'].values
labels = dataset['sentiment'].values

# 라벨을 One-Hot 인코딩으로 변환
label_map = {'부정': 0, '긍정': 1}
labels = np.array([label_map[label] for label in labels])

In [ ]:
# 문장 토크나이징 길이 시각화, 원하는 비율을 커버하는 최적의 padding길이 return
padding_length = sent_length_vis(sentences, modelpath, 0.95) # 문장 리스트, 토크나이저 경로, 원하는 비율
print("최적의 padding_length는 '%s' 입니다."%padding_length)

In [5]:
train_dataloader, val_dataloader = dataset_split(padding_length, sentences, labels, modelpath, 0.2)

100%|██████████| 9958/9958 [00:02<00:00, 4824.25it/s]


In [ ]:
model, optimizer, loss_fn, device = load_model(modelpath, 2) # 모델경로, label수 - (필수) 모델 dict 경로, 옵티마이저 dict 경로, 모델 체크포인트 경로 - (선택사항)

In [ ]:
train_losses_epoch, train_accs_epoch, val_losses_epoch, val_accs_epoch = train_with_early_stopping(train_dataloader, val_dataloader, model, optimizer, loss_fn, device, 5,  patience=3)

In [8]:
plot_training_progress(train_losses_epoch, train_accs_epoch, val_losses_epoch, val_accs_epoch)